In [ ]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import mr

In [ ]:
from mrcnn import model as modellib
from mrcnn import visualize
from mrcnn import utils

In [ ]:
work_path = os.path.join("E:", os.sep, f"RCNN{mr.CLASS_NAME}{mr.IMAGE_DIM}Valid")
os.chdir(work_path)

In [ ]:
inference_config = mr.InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir="logs")

find_last = False
if find_last:
    model_path = model.find_last()
    print("Loading weights from ", model_path)
else:
    model_path = "mask_rcnn_mr_0500.h5"
model.load_weights(model_path, by_name=True)

In [ ]:
dataset = mr.MRDataset()
dataset.load(work_path)
dataset.prepare()

In [ ]:
def compute_ap(image_id):
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset,
                                                                              inference_config,
                                                                              image_id,
                                                                              use_mini_mask=False)
    results = model.detect([image], verbose=0)
    r = results[0]
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    return AP
    
image_ids = dataset.image_ids.copy()
random.shuffle(image_ids)
image_ids = image_ids[:100]
APs = [compute_ap(image_id) for image_id in image_ids]
print("mAP: ", np.mean(APs))